In [1]:
!pip install pyspark==3.4.0 findspark pyarrow fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317123 sha256=9de1a44e8e292cf942abd1ceb848e0a548806f4cdb4f1cfb51adb96b88c31385
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
# Import findspark and initialise.
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [7]:

# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT
  YEAR(date) as Year,
  ROUND(AVG(price),2) as Average_Price
FROM home_sales
WHERE bedrooms =4
GROUP BY YEAR(date)
ORDER BY YEAR(date) desc
""").show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [9]:

# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""
SELECT
  date_built as Year,
  ROUND(AVG(price),2) as  Average_Price
FROM home_sales
WHERE bedrooms = 3 and bathrooms = 3
GROUP BY date_built
ORDER BY date_built desc
""").show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2017|    292676.79|
|2016|    290555.07|
|2015|     288770.3|
|2014|    290852.27|
|2013|    295962.27|
|2012|    293683.19|
|2011|    291117.47|
|2010|    292859.62|
+----+-------------+



In [10]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT
  date_built as Year_Built,
  ROUND(AVG(price),2) as  Average_Price
FROM home_sales
WHERE bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built desc
""").show()

+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT
  view,
  ROUND(AVG(price),2) as Average_Price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.647047519683838 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT
  view,
  ROUND(AVG(price),2) as Average_Price
FROM home_sales
GROUP BY view
HAVING AVG(price) < 350000
ORDER BY view asc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
|   0|    285069.21|
|   1|    288248.11|
|  10|    289693.49|
|  11|    280356.07|
|  12|    286575.22|
|  13|    285849.95|
|  14|    282519.32|
|  15|    284907.04|
|  16|    291990.83|
|  17|    286601.22|
|  18|    287532.36|
|  19|    280347.62|
|   2|    287710.06|
|  20|    284820.43|
|  21|    283451.36|
|  22|    284908.42|
|  23|    286977.37|
|  24|    281878.86|
|  25|    288583.21|
|  26|    281224.95|
+----+-------------+
only showing top 20 rows

--- 0.8507857322692871 seconds ---


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('date_built_partitioned')

In [16]:
# 11. Read the parquet formatted data.
date_built_partitioned_df = spark.read.parquet('date_built_partitioned')

In [17]:
# 12. Create a temporary table for the parquet data.
date_built_partitioned_df.createOrReplaceTempView('home_sales_partitioned')

In [18]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT
  view,
  ROUND(AVG(price),2) as Average_Price
FROM home_sales_partitioned
GROUP BY view
HAVING AVG(price) < 350000
ORDER BY view asc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
|   0|    285069.21|
|   1|    288248.11|
|  10|    289693.49|
|  11|    280356.07|
|  12|    286575.22|
|  13|    285849.95|
|  14|    282519.32|
|  15|    284907.04|
|  16|    291990.83|
|  17|    286601.22|
|  18|    287532.36|
|  19|    280347.62|
|   2|    287710.06|
|  20|    284820.43|
|  21|    283451.36|
|  22|    284908.42|
|  23|    286977.37|
|  24|    281878.86|
|  25|    288583.21|
|  26|    281224.95|
+----+-------------+
only showing top 20 rows

--- 1.251495599746704 seconds ---


In [19]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [20]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False

In [22]:
# 16. Another check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is still cached')
else:
  print('all clear')

all clear
